In [5]:
## turn on the autocomplete if off by default
%config Completer.use_jedi = False

## Below is a simple NN example in PyTorch to illustrate:
1. Data Read (interactive)
2. Normalize feature set



In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
# Model Report Card
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

In [3]:
from tqdm import tqdm, tqdm_notebook

In [6]:
# upload iris data interactively to google drive and access the data
# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# access data file
# df = pd.read_csv(io.StringIO(uploaded['iris.csv'].decode('utf-8')))
df = pd.read_csv("/content/gdrive/MyDrive/Solution Implementation Course/loans_2007.csv")
df.drop(df.columns[[0]], axis=1, inplace=True)# there is an unnamed column in the first col.
df.head(4)

In [ ]:
df.describe()

In [ ]:
df.verification_status.value_counts()

In [ ]:
# what isthe structure of my dataset
print(df.shape)
# df.describe()

In [ ]:
aggegate = df.loan_status.value_counts().to_frame('n')
aggegate['percent'] = np.round(100*(aggegate['n']/(aggegate['n'].sum())),2)
aggegate

1. Remove all the columns that get added after loan issue
> amount of loan <br/>
> installment
2. Any other variable?

In [ ]:
df.drop(['loan_amnt', 'installment', 'term'], axis=1, inplace=True)
# df.drop(['installment', 'term'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
# Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df.isnull().sum()/df.shape[0])

In [ ]:
df = df.dropna(subset=['revol_util'])

In [ ]:
# Make category of columns

# Numerical variables ( can be continous (can take any numerical value) or discreet (can only take whole numbers as values) )
# numerical_column = [ 'int_rate', 'emp_length', 'annual_inc', \
#                     'zip_code', 'dti', 'delinq_2yrs', 'fico_range_high', \
#                     'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',\
#                     'revol_util', 'total_acc', 'last_fico_range_high']
numerical_column = [ 'int_rate', 'emp_length', 'annual_inc', 'delinq_2yrs', 'fico_range_high','revol_bal', 'open_acc','loan_status']
# Categorical - ( can be nominal (no order categories) or ordinal (ordered categories) )
# categorical_column = ['home_ownership', 'verification_status', 'purpose']

target_column = ['loan_status']

In [ ]:
numerical_column[:-1]

In [ ]:
# Split into feature set "X" and target variable "y"
y = df[target_column]
df = df[numerical_column]
X = df.drop(columns=target_column)
print(f"Shape of X: {X.shape}\nShape of y: {y.shape}")

In [ ]:
# Split data into train and test (Validation in our case)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1, test_size=0.30)

print(f"Shape of Xtrain: {Xtrain.shape}\n\
Shape of Xtest: {Xtest.shape}\n\
Shape of ytrain: {ytrain.shape}\n\
Shape of ytest: {ytest.shape}")

In [ ]:
# # Data transformation

# # Scaling choosen as it bring data into range of 0 to 1 
# ct = ColumnTransformer(transformers=[('standardscaler', StandardScaler(), numerical_column[:-1])])
# # ct = ColumnTransformer(transformers=[('standardscaler', StandardScaler(), numerical_column),
# #                                 ('onehotencoder', OneHotEncoder(drop="first"), categorical_column)])
# # Apply on Train features
# ct.fit(Xtrain) # Learn the stats for this data set
# Xtrain = ct.transform(Xtrain) # Apply Transformations

# # Second way in one line
# # ct.fit_transform(Xtrain)

# # Apply on Test Features (Make sure you dont use Fit this time)
# # You will apply the same stats for transfromation over test set which you have have learned over the training set.

# Xtest = ct.transform(Xtest)

# # Check
# print(f"Shape of Xtrain: {Xtrain.shape}\n\
# Shape of Xtest: {Xtest.shape}\n\
# Shape of ytrain: {ytrain.shape}\n\
# Shape of ytest: {ytest.shape}")

In [ ]:
# Here our X is in correct 2-d shape as a matrix
# Convert y to 1-d array

ytrain = ytrain.values.ravel()
ytest = ytest.values.ravel()

In [ ]:
# Now the train a Logistics Regression Model

from sklearn.ensemble import RandomForestClassifier
# model=RandomForestClassifier(class_weight='balanced')
model = LogisticRegression(solver='newton-cg', class_weight='balanced')              # 1. instantiate model (Again you can sepcify the model specific Hyperparameters)

model.fit(Xtrain, ytrain)                  # 2. fit model to data

y_model = model.predict(Xtest)             # 3. predict on new data
y_model

In [ ]:
# Create a new DataFrame to store the results
classification_results = pd.DataFrame(columns=['classifier_Name', 'Accuracy_Score'])

In [ ]:
# Make sure you pass the values in same order. First ytest (Orginal values) and then y_model (Model which will predict the y values)
accuracy_score(ytest, y_model)
classification_results = classification_results.append({'classifier_Name': 'Random forest', 'Accuracy_Score': accuracy_score(ytest, y_model)}, ignore_index=True)
classification_results

In [ ]:
# Plot Confusion Matrix
# Format is -> confusion_matrix(Actual, Predicted)
c_matrix = confusion_matrix(ytest, y_model)
sns.heatmap(c_matrix.T, square=True, annot=True, fmt='d', cbar=False)
plt.title('Naive Bayes Confusion Matrix')
plt.xlabel('true label')
plt.ylabel('predicted label')

In [ ]:
print(classification_report(ytest, y_model))

In [ ]:
# save model in a pickle format .pkl
import pickle
pickle_out = open("/content/gdrive/MyDrive/Solution Implementation Course/model.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()